In [72]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py
import tensorflow as tf
import os

from src.utils import to_rgb
from src.satellite_images.storage import SentinelDataset, SentinelDatasetIterator
from src.mask.mask_dataset import MaskDataset, MaskDatasetIterator
from src.mask.utils import apply_mask_to_image_series, apply_mask_to_image

%load_ext autoreload
%autoreload 2

data_path = '../../../kornmo-data-files/raw-data/crop-classification-data/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
print("Reading data")
data = gpd.read_file(f"{data_path}/training_data.gpkg")
data['orgnr'] = data['orgnr'].astype(int)
data['year'] = data['year'].astype(int)
print("Reading masks")
masks = MaskDataset(f"{data_path}/train_data_masks.h5")
print("Reading satellite images")
satellite_imgs = SentinelDataset('E:/MasterThesisData/Satellite_Images/satellite_images_train.h5')



Reading data
Reading masks
Reading satellite images


In [74]:
data = data.loc[data['planted'] != 'erter']
data = data.loc[data['planted'] != 'rughvete']
data = data.loc[data['planted'] != 'oljefro']
data = data.loc[data['planted'] != 'rug']

# print(969923173 in data['orgnr'].unique())
# print(2017 in data.loc[data['orgnr'] == 969923173]['year'].unique())
# label = data.loc[data['orgnr'] == 969923173]
# print(label)

In [75]:

labels = list(set(data['planted']))
print(labels)
print(pd.Series(list(data['planted'])).value_counts())
def add_labels(orgnr, year, data_arg):
    orgnr = int(orgnr)
    year = int(year)
    if orgnr in data['orgnr'].unique() and year in data.loc[data['orgnr'] == orgnr]['year'].unique():
        label = data.loc[data['orgnr'] == orgnr].loc[data['year'] == year]['planted'].iloc[0]
        index = labels.index(label)
        arr = [0 for _ in range(0, len(labels))]
        arr[index] = 1
        return {'class': arr}


['hvete', 'bygg', 'havre']
bygg     126957
havre     32227
hvete     29149
dtype: int64


In [76]:
train, val = satellite_imgs.to_iterator().split(rand_seed='corn')

train = train.with_data(add_labels)
val = val.with_data(add_labels)
masks_it = masks.get_iterator()
mask_dict = {}

for orgnr, year, mask in masks_it:
    mask_dict[f'{orgnr}/{year}'] = mask


train = train.filter(lambda orgnr, year, _,__: f"{orgnr}/{year}" in mask_dict)
val = val.filter(lambda orgnr, year, _,__: f"{orgnr}/{year}" in mask_dict)

print(f"train samples: {len(train)}")
print(f"val samples: {len(val)}")

#7737, 1937

train samples: 7737
val samples: 1937


In [80]:
import random

def train_generator():
    for orgnr, year, imgs, label in train:
        for img in imgs:
            img = apply_mask_to_image(mask_dict[f'{orgnr}/{year}'], img)
            yield img, label['class']


def val_generator():
    for orgnr, year, imgs, label in val:
        for img in imgs:
            img = apply_mask_to_image(mask_dict[f'{orgnr}/{year}'], img)
            yield img, label['class']



In [83]:

train_dataset = tf.data.Dataset.from_generator(
    train_generator,
    output_types=(tf.dtypes.float64, tf.dtypes.int64),
    output_shapes=((100, 100, 12), 3)
)

val_dataset = tf.data.Dataset.from_generator(
    val_generator,
    output_types=(tf.dtypes.float64, tf.dtypes.int64),
    output_shapes=((100, 100, 12), 3)
)

# print(f"training samples: {len(train_dataset)}")
# print(f"Validation samples: {len(val_dataset)}")

In [92]:
from keras import models
from keras.applications.densenet import layers
from keras.models import load_model
from tensorflow import optimizers
from src.kornmo.pyimagesearch.callbacks.epochcheckpoint import EpochCheckpoint
from src.kornmo.pyimagesearch.callbacks.trainingmonitor import TrainingMonitor

plotPath = os.path.sep.join(["training", "hybrid_more_features.png"])
jsonPath = os.path.sep.join(["training", "hybrid_more_features.json"])
callbacks = [
	EpochCheckpoint('./training', every=1, startAt=0),
	TrainingMonitor(plotPath, jsonPath=jsonPath, startAt=0)
]

restart = True
if restart:
    input_layer = layers.Input(shape=(100, 100, 12))
    cnn = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_layer)
    cnn = layers.MaxPooling2D((2, 2))(cnn)
    cnn = layers.Conv2D(32, (3, 3), strides=(2,2), activation='relu', padding='same')(cnn)
    cnn = layers.MaxPooling2D((2, 2))(cnn)
    cnn = layers.Conv2D(64, (3, 3), strides=(2,2), activation='relu', padding='same')(cnn)
    cnn = layers.MaxPooling2D((2, 2))(cnn)
    cnn = layers.Flatten()(cnn)
    cnn = layers.Dense(128, activation="relu")(cnn)
    cnn = layers.Dropout(0.2)(cnn)
    cnn = layers.Dense(3, activation='softmax')(cnn)




    cnn = models.Model(inputs=[input_layer], outputs=cnn, name="cnn_pure")
    cnn.compile(
        optimizer=optimizers.Adam(),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['categorical_accuracy']
    )
    history = cnn.fit(
        train_dataset.batch(32).prefetch(2),
        validation_data=val_dataset.batch(32).prefetch(2),
        epochs=100,
        callbacks=callbacks
    )
else:

    cnn_net = load_model('./training/epoch_4.hdf5')

    cnn_history = cnn_net.fit(
        train_dataset.take(10000).batch(32).prefetch(2),
        validation_data=val_dataset.batch(32).prefetch(2),
        epochs=100,
        verbose=1,
        callbacks=callbacks
    )


Epoch 1/100
7254/7254 [==============================] - 2015s 278ms/step - loss: 0.9480 - categorical_accuracy: 0.5971 - val_loss: 0.9685 - val_categorical_accuracy: 0.5834
Epoch 2/100
7254/7254 [==============================] - 1977s 272ms/step - loss: 0.9400 - categorical_accuracy: 0.5970 - val_loss: 0.9572 - val_categorical_accuracy: 0.5834
Epoch 3/100
7254/7254 [==============================] - 2167s 299ms/step - loss: 0.9323 - categorical_accuracy: 0.5970 - val_loss: 0.9592 - val_categorical_accuracy: 0.5831
Epoch 4/100
7254/7254 [==============================] - 2170s 299ms/step - loss: 0.9180 - categorical_accuracy: 0.5987 - val_loss: 0.9765 - val_categorical_accuracy: 0.5685
Epoch 5/100
7254/7254 [==============================] - 2183s 301ms/step - loss: 0.8901 - categorical_accuracy: 0.6075 - val_loss: 0.9525 - val_categorical_accuracy: 0.5710
Epoch 6/100
7254/7254 [==============================] - 2216s 306ms/step - loss: 0.8445 - categorical_accuracy: 0.6269 - val_loss

KeyboardInterrupt: 